In [ ]:
import torch
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchsummary import summary

In [ ]:
#DEFINE YOUR DEVICE
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device) #if cpu, go Runtime-> Change runtime type-> Hardware accelerator GPU -> Save -> Redo previous steps

In [ ]:
#DOWNLOAD CIFAR-10 DATASET
train_data = datasets.CIFAR10('./data', train = True, download = True, transform = transforms.ToTensor())

test_data = datasets.CIFAR10('./data', train = False, transform = transforms.ToTensor())

In [ ]:
#DEFINE DATA GENERATOR
batch_size = 100
train_generator = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)

test_generator = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = False)

In [ ]:
#DEFINE NEURAL NETWORK MODEL
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = torch.nn.Conv2d(3, 32, kernel_size = 3, stride = 1)
    self.conv2 = torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1)
    self.conv3 = torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1)
    self.conv4 = torch.nn.Conv2d(128,256, kernel_size = 3, stride = 1)

    self.mpool = torch.nn.MaxPool2d(2)

    self.fc1 = torch.nn.Linear(1024, 2048)  # Increased output size to 1024
    self.fc2 = torch.nn.Linear(2048, 256)  # Increased output size to 256
    self.fc3 = torch.nn.Linear(256, 10)

    self.relu = torch.nn.ReLU()
    self.sigmoid = torch.nn.Sigmoid()
    self.drop = torch.nn.Dropout(0.4)

    self.bn1 =  torch.nn.BatchNorm2d(32)
    self.bn2 =  torch.nn.BatchNorm2d(64)
    self.bn3 =  torch.nn.BatchNorm2d(128)
    self.bn4 =  torch.nn.BatchNorm2d(256)
  def forward(self, x):
    hidden = self.bn1(self.relu(self.conv1(x)))
    hidden = self.mpool(self.bn2(self.relu(self.conv2(hidden))))
    hidden = self.mpool(self.bn3(self.relu(self.conv3(hidden))))
    hidden = self.mpool(self.bn4(self.relu(self.conv4(hidden))))
    hidden = hidden.view(-1,1024)
    hidden = self.relu(self.fc1(hidden))
    hidden = self.drop(hidden)
    hidden = self.relu(self.fc2(hidden))
    hidden = self.drop(hidden)
    output = self.fc3(hidden)
    return output

In [ ]:
#CREATE MODEL
model = CNN()
model.to(device)
summary(model,(3,32,32))

In [ ]:
#DEFINE LOSS FUNCTION AND OPTIMIZER
learning_rate = 0.001

loss_fun = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
#TRAIN THE MODEL
model.train()
epoch = 10

num_of_batch=np.int32(len(train_generator.dataset)/batch_size)

loss_values = np.zeros(epoch*num_of_batch)
for i in range(epoch):
  for batch_idx, (x_train, y_train) in enumerate(train_generator):
    x_train, y_train = x_train.to(device), y_train.to(device)
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss = loss_fun(y_pred, y_train)
    loss_values[num_of_batch*i+batch_idx] = loss.item()
    loss.backward()
    optimizer.step()
    if (batch_idx+1) % batch_size == 0:
        print('Epoch: {}/{} [Batch: {}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            i+1, epoch, (batch_idx+1) * len(x_train), len(train_generator.dataset),
            100. * (batch_idx+1) / len(train_generator), loss.item()))

In [ ]:
#PLOT THE LEARNING CURVE
iterations = np.linspace(0,epoch,num_of_batch*epoch)
plt.plot(iterations, loss_values)
plt.title('Learning Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid('on')

In [ ]:
#TEST THE MODEL
model.eval()
correct=0
total=0

for x_val, y_val in test_generator:
  x_val = x_val.to(device)
  y_val = y_val.to(device)

  output = model(x_val)
  y_pred = output.argmax(dim=1)

  for i in range(y_pred.shape[0]):
    if y_val[i]==y_pred[i]:
      correct += 1
    total +=1

print('Validation accuracy: %.2f%%' %((100*correct)//(total)))